In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
from skimage.transform import resize as imresize
import pickle
import time
import sys


In [3]:
training_folder = "train_images"
testing_folder = "test_images"
batch_train = 20
batch_test = 25

In [8]:
def get_vgg_model():
#     download('https://s3.amazonaws.com/cadl/models/vgg16.tfmodel')
    with open("vgg16.tfmodel", mode='rb') as f:
        graph_def = tf.GraphDef()           # makes a graph defintion using function tf.GraphDef()
        try:
            graph_def.ParseFromString(f.read())     # feeds the vgg16 model onto the graph definition
        except:
            print('try adding PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python ' +
                  'to environment.  e.g.:\n' +
                  'PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python ipython\n' +
                  'See here for info: ' +
                  'https://github.com/tensorflow/tensorflow/issues/582')

    # download('https://s3.amazonaws.com/cadl/models/synset.txt')
    # with open('synset.txt') as f:
    #     labels = [(idx, l.strip()) for idx, l in enumerate(f.readlines())]

    return {
        'graph_def': graph_def
        # 'labels': labels
        # 'preprocess': preprocess,
        # 'deprocess': deprocess
    }

In [5]:
def preprocess(img, crop=True, resize=True, dsize=(224, 224)):
    if img.dtype == np.uint8:
        img = img / 255.0

    if crop:
        short_edge = min(img.shape[:2])
        yy = int((img.shape[0] - short_edge) / 2)
        xx = int((img.shape[1] - short_edge) / 2)
        crop_img = img[yy: yy + short_edge, xx: xx + short_edge]
    else:
        crop_img = img

    if resize:
        norm_img = imresize(crop_img, dsize, preserve_range=True)
    else:
        norm_img = crop_img

    return (norm_img).astype(np.float32)

def deprocess(img):
    return np.clip(img * 255, 0, 255).astype(np.uint8)
    # return ((img / np.max(np.abs(img))) * 127.5+127.5).astype(np.uint8)


In [9]:
net = get_vgg_model()

AttributeError: module 'tensorflow' has no attribute 'GraphDef'

In [ ]:
g1 = tf.Graph()                                                 # initializes a tf graph

with tf.Session(graph=g1) as sess, g1.device('/cpu:0'):         # '/cpu:0' specifies the first CPU available in the system.
    tf.import_graph_def(net['graph_def'], name='vgg')          #uploads the graph from vgg16 onto the g1 graph and names it vgg


In [ ]:
def get_content_feature(img_4d):
# gets the content features from maxpool5 layer of vgg16 and feeds in data img_4d via placeholder x1.

    with tf.Session(graph=g1) as sess, g1.device('/gpu:0'):


            content_layer = 'vgg/pool5:0'
            content_features= g1.get_tensor_by_name(content_layer).eval(
                    session=sess,
                    feed_dict={x1: img_4d
                    })

            # train_new.append(content_features)
            print(content_features.shape)
            return content_features